In [14]:
import numpy as np
from scipy.linalg import expm
from itertools import product

# --- Pauli matrices and utilities ---
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

def kron_n(*ops):
    return np.kron(np.kron(np.kron(ops[0], ops[1]), ops[2]), ops[3])

# --- Operators for building H ---
def number_op(k):
    ops = [I] * 4
    ops[k] = 0.5 * (I + Z)
    return kron_n(*ops)

def SP(k):
    ops = [I] * 4
    ops[k] = 0.5 * (X - 1j * Y)
    return kron_n(*ops)

def SM(k):
    ops = [I] * 4
    ops[k] = 0.5 * (X + 1j * Y)
    return kron_n(*ops)

def sigma_anyonic(k):
    ops = [I] * 4
    ops[k] = Z
    return kron_n(*ops)

def jw_string(path):
    if not path:
        return np.eye(16)
    result = sigma_anyonic(path[0])
    for k in path[1:]:
        result = result @ sigma_anyonic(k)
    return result

# --- Hamiltonian with φ = π/2 ---
phi = np.pi / 2
AB_phase = np.exp(1j * phi)
edges = [(0, 1), (1, 2), (1, 3), (0, 2), (2, 3)]
red_link = (1, 2)
path_dict = {(0, 1): [], (1, 2): [], (1, 3): [], (0, 2): [1], (2, 3): []}

H = np.zeros((16, 16), dtype=complex)
for (i, j) in edges:
    path = path_dict[(i, j)]
    t = AB_phase if (i, j) == red_link or (j, i) == red_link else 1.0
    term = SP(i) @ jw_string(path) @ SM(j)
    H += t * term + np.conj(t) * term.conj().T

# Add density-density interactions
for i in range(4):
    for j in range(i + 1, 4):
        H += number_op(i) @ number_op(j)

# --- Define time evolution ---
T = 1.0
U_target = expm(-1j * H * T)

In [15]:
import numpy as np
from scipy.linalg import expm
from scipy.optimize import minimize
import random

# Pauli matrices and identity
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, 1]])

# Utility
def kron_n(a, b, c, d):
    return np.kron(np.kron(np.kron(a, b), c), d)

# Native gate constructors
def Rx(k, theta):
    ops = [I] * 4
    ops[k] = expm(-1j * theta * X / 2)
    return kron_n(*ops)

def Rz(k, theta):
    ops = [I] * 4
    ops[k] = expm(-1j * theta * Z / 2)
    return kron_n(*ops)

def MS(i, j, theta):
    ops = [I] * 4
    ops[i] = X
    ops[j] = X
    return expm(-1j * theta * kron_n(*ops))

def ZZ(i, j, theta):
    ops = [I] * 4
    ops[i] = Z
    ops[j] = Z
    return expm(-1j * theta * kron_n(*ops))

# Gate rotation utility
def rotate_tile_gates(gates, degrees):
    shift = (degrees // 90) % 4
    return gates[shift:] + gates[:shift]

# Rotatable tile definitions
def tile_I(theta, rotation=0):
    return rotate_tile_gates([MS(0, 1, theta[0]), MS(2, 3, theta[1]), Rx(0, theta[2]), Rz(0, theta[3])], rotation)

def tile_O(theta, rotation=0):
    return rotate_tile_gates([MS(0, 1, theta[0]), ZZ(0, 1, theta[1]), Rx(0, theta[2]), Rz(0, theta[3])], rotation)

def tile_L(theta, rotation=0):
    return rotate_tile_gates([Rx(0, theta[0]), MS(0, 1, theta[1]), Rz(1, theta[2]), ZZ(1, 2, theta[3])], rotation)

def tile_J(theta, rotation=0):
    return rotate_tile_gates([Rx(3, theta[0]), MS(2, 3, theta[1]), Rz(2, theta[2]), ZZ(1, 2, theta[3])], rotation)

def tile_T(theta, rotation=0):
    return rotate_tile_gates([MS(0, 2, theta[0]), MS(1, 2, theta[1]), Rx(2, theta[2]), ZZ(2, 3, theta[3])], rotation)

def tile_Z(theta, rotation=0):
    return rotate_tile_gates([MS(0, 1, theta[0]), MS(1, 2, theta[1]), Rx(2, theta[2]), Rz(3, theta[3])], rotation)

def tile_S(theta, rotation=0):
    return rotate_tile_gates([MS(1, 2, theta[0]), MS(2, 3, theta[1]), Rx(1, theta[2]), Rz(0, theta[3])], rotation)

# Tile dictionary with rotation support
tetromino_tiles = {
    'I': tile_I,
    'O': tile_O,
    'L': tile_L,
    'J': tile_J,
    'T': tile_T,
    'Z': tile_Z,
    'S': tile_S,
}

# Build circuit from rotated tetrominoes
def build_rotated_tetromino_circuit(tile_configs, params):
    U = np.eye(16, dtype=complex)
    for i, (name, rotation) in enumerate(tile_configs):
        theta_block = params[4 * i : 4 * (i + 1)]
        for gate in tetromino_tiles[name](theta_block, rotation):
            U = gate @ U
    return U

# MAIN OPTIMIZATION LOOP with random rotation layout
def main():
    # Define your actual target unitary here
    #U_target = np.eye(16, dtype=complex)  # placeholder

    # Random tile layout and rotations
    tile_names = ['I', 'O', 'T', 'Z', 'S', 'L', 'J']
    tile_configs = [(random.choice(tile_names), random.choice([0, 90, 180, 270])) for _ in range(5)]
    print("Tile layout:", tile_configs)

    init_params = [np.pi / 4] * (4 * len(tile_configs))

    def loss(params):
        U_trial = build_rotated_tetromino_circuit(tile_configs, params)
        fidelity = np.abs(np.trace(U_trial.conj().T @ U_target))**2 / 16**2
        return 1 - fidelity

    result = minimize(loss, init_params, method='Nelder-Mead', options={'maxiter': 500})

    print("\n--- Optimization Complete ---")
    print("Fidelity:", 1 - result.fun)
    print("Optimized Parameters:", result.x)

if __name__ == "__main__":
    main()


Tile layout: [('Z', 180), ('O', 90), ('S', 0), ('O', 180), ('O', 90)]

--- Optimization Complete ---
Fidelity: 0.031019551581119242
Optimized Parameters: [ 0.75252244  1.11847502  0.96183012  1.4164584   0.38033058  1.31598506
  0.10323302  2.19342072 -0.4457366   0.76929846  0.84736411  0.89331403
  0.29051659  0.83390221  0.46188377  0.40924298  0.10581684  0.93663752
  0.39389597  1.2385321 ]


In [24]:
import numpy as np
from scipy.linalg import expm
from scipy.optimize import minimize
import random

# Pauli matrices and identity
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, 1]])

# Tensor utility
def kron_n(a, b, c, d):
    return np.kron(np.kron(np.kron(a, b), c), d)

# Define gates
def Rx(k, theta):
    ops = [I] * 4
    ops[k] = expm(-1j * theta * X / 2)
    return kron_n(*ops)

def Rz(k, theta):
    ops = [I] * 4
    ops[k] = expm(-1j * theta * Z / 2)
    return kron_n(*ops)

def MS(i, j, theta):
    ops = [I] * 4
    ops[i] = X
    ops[j] = X
    return expm(-1j * theta * kron_n(*ops))

def ZZ(i, j, theta):
    ops = [I] * 4
    ops[i] = Z
    ops[j] = Z
    return expm(-1j * theta * kron_n(*ops))

# Rotation logic for gate sequence
def rotate_tile_gates(gates, degrees):
    shift = (degrees // 90) % 4
    return gates[shift:] + gates[:shift]

# Rotatable tetromino tiles
def tile_I(theta, rotation=0):
    return rotate_tile_gates([MS(0, 1, theta[0]), MS(2, 3, theta[1]), Rx(0, theta[2]), Rz(0, theta[3])], rotation)

def tile_O(theta, rotation=0):
    return rotate_tile_gates([MS(0, 1, theta[0]), ZZ(0, 1, theta[1]), Rx(0, theta[2]), Rz(0, theta[3])], rotation)

def tile_L(theta, rotation=0):
    return rotate_tile_gates([Rx(0, theta[0]), MS(0, 1, theta[1]), Rz(1, theta[2]), ZZ(1, 2, theta[3])], rotation)

def tile_J(theta, rotation=0):
    return rotate_tile_gates([Rx(3, theta[0]), MS(2, 3, theta[1]), Rz(2, theta[2]), ZZ(1, 2, theta[3])], rotation)

def tile_T(theta, rotation=0):
    return rotate_tile_gates([MS(0, 2, theta[0]), MS(1, 2, theta[1]), Rx(2, theta[2]), ZZ(2, 3, theta[3])], rotation)

def tile_Z(theta, rotation=0):
    return rotate_tile_gates([MS(0, 1, theta[0]), MS(1, 2, theta[1]), Rx(2, theta[2]), Rz(3, theta[3])], rotation)

def tile_S(theta, rotation=0):
    return rotate_tile_gates([MS(1, 2, theta[0]), MS(2, 3, theta[1]), Rx(1, theta[2]), Rz(0, theta[3])], rotation)

tetromino_tiles = {
    'I': tile_I,
    'O': tile_O,
    'L': tile_L,
    'J': tile_J,
    'T': tile_T,
    'Z': tile_Z,
    'S': tile_S,
}

# Build unitary from tile list with rotations
def build_rotated_tetromino_circuit(tile_configs, params):
    U = np.eye(16, dtype=complex)
    for i, (name, rotation) in enumerate(tile_configs):
        theta_block = params[4 * i : 4 * (i + 1)]
        for gate in tetromino_tiles[name](theta_block, rotation):
            U = gate @ U
    return U

# === Build the target unitary U = exp(-i H t) ===
def build_target_unitary():
    def number_op(k):
        ops = [I] * 4
        ops[k] = 0.5 * (I + Z)
        return kron_n(*ops)

    def SP(k):
        ops = [I] * 4
        ops[k] = 0.5 * (X - 1j * Y)
        return kron_n(*ops)

    def SM(k):
        ops = [I] * 4
        ops[k] = 0.5 * (X + 1j * Y)
        return kron_n(*ops)

    def sigma_anyonic(k):
        ops = [I] * 4
        ops[k] = Z
        return kron_n(*ops)

    def jw_string(path):
        if not path:
            return np.eye(16)
        result = sigma_anyonic(path[0])
        for k in path[1:]:
            result = result @ sigma_anyonic(k)
        return result

    phi = np.pi / 2
    AB_phase = np.exp(1j * phi)
    edges = [(0, 1), (1, 2), (1, 3), (0, 2), (2, 3)]
    red_link = (1, 2)
    path_dict = {(0, 1): [], (1, 2): [], (1, 3): [], (0, 2): [1], (2, 3): []}

    H = np.zeros((16, 16), dtype=complex)
    for (i, j) in edges:
        path = path_dict[(i, j)]
        t = AB_phase if (i, j) == red_link or (j, i) == red_link else 1.0
        term = SP(i) @ jw_string(path) @ SM(j)
        H += t * term + np.conj(t) * term.conj().T

    for i in range(4):
        for j in range(i + 1, 4):
            H += number_op(i) @ number_op(j)

    return expm(-1j * H * 1.0)

# Run optimization
def main_rotated_tetromino_optimization(U_target):
    tile_names = list(tetromino_tiles.keys())
    tile_configs = [(random.choice(tile_names), random.choice([0, 90, 180, 270])) for _ in range(5)]
    print("Tile layout:", tile_configs)

    init_params = [np.pi / 4] * (4 * len(tile_configs))

    def loss(params):
        U_trial = build_rotated_tetromino_circuit(tile_configs, params)
        fidelity = np.abs(np.trace(U_trial.conj().T @ U_target))**2 / 16**2
        return 1 - fidelity

    result = minimize(loss, init_params, method='Nelder-Mead', options={'maxiter': 500})
    final_fidelity = 1 - result.fun

    print("\n--- Optimization Complete ---")
    print("Fidelity:", final_fidelity)
    print("Optimized Parameters:", result.x)

    return tile_configs, final_fidelity, result.x

# Run the whole process
# if __name__ == "__main__":
#     U_target = build_target_unitary()
#     main_rotated_tetromino_optimization(U_target)


def run_batch_optimizations(U_target, N=20):
    best_fid = 0
    best_config = None
    best_params = None

    for i in range(N):
        print(f"\n>>> Trial {i+1}/{N}")
        config, fid, params = main_rotated_tetromino_optimization(U_target)
        if fid > best_fid:
            best_fid = fid
            best_config = config
            best_params = params

    print("\n=== BEST OVERALL RESULT ===")
    print("Best Fidelity:", best_fid)
    print("Best Tile Layout:", best_config)
    print("Best Parameters:", best_params)

    return best_fid, best_config, best_params

# Run it
if __name__ == "__main__":
    U_target = build_target_unitary()
    run_batch_optimizations(U_target, N=20)



>>> Trial 1/20
Tile layout: [('J', 180), ('Z', 270), ('L', 90), ('L', 270), ('L', 0)]

--- Optimization Complete ---
Fidelity: 0.10308228435247035
Optimized Parameters: [ 0.89553776  0.47286699  0.53448915  0.90914083  0.84577489 -0.03288158
  0.02981493  1.24122841 -0.06405472  1.99600912  2.36330487  0.68051707
  0.79034152  0.65718985  1.63201944  0.87778681 -0.43487071  0.1670243
  1.43956632  0.45150146]

>>> Trial 2/20
Tile layout: [('S', 180), ('J', 270), ('Z', 90), ('Z', 270), ('L', 90)]

--- Optimization Complete ---
Fidelity: 0.1061372622448854
Optimized Parameters: [ 0.79051219  0.28705582  1.54055075  0.66738095  0.68227461 -0.05507074
  1.08232253  0.49458255  1.37086377  0.88035912  0.56665324  0.32878038
  0.89759056  0.69222841  0.99123976  0.7203848   0.85452901  0.99822675
  0.82285094  0.95088482]

>>> Trial 3/20
Tile layout: [('S', 180), ('O', 90), ('S', 270), ('J', 0), ('O', 180)]

--- Optimization Complete ---
Fidelity: 0.12989064755482627
Optimized Parameters: [